In [ ]:
#James Chartouni
#code edited/inspired https://www.kaggle.com/samratp/aggregates-sumvalues-sumzeros-k-means-pca
#code edited/inspired https://www.kaggle.com/samratp/aggregates-sumvalues-sumzeros-k-means-pca
import pandas as pd
import numpy as np
import gc

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import RobustScaler, StandardScaler, normalize, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD, PCA, SparsePCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AffinityPropagation, MeanShift, DBSCAN, Birch, SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.linear_model import Lasso, Ridge, ElasticNet


import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot

from helpers import *
from scipy.stats import boxcox
from skopt import BayesSearchCV
from skopt import gp_minimize

In [2]:
train = pd.read_csv("data/train_cleaned.csv")
train_orig = pd.read_csv("data/train.csv")
y = np.log1p(train_orig["target"].values)
X = train.values

print ("Rows: " + str(train.shape[0]) + ", Columns: " + str(train.shape[1]))
#print(train.head())

#test = pd.read_csv('data/test_cleaned.csv')
#test_X = test.drop(["ID"], axis=1)
#print ("Rows: " + str(test.shape[0]) + ", Columns: " + str(test.shape[1]))

#print(test.head())

Rows: 4459, Columns: 1420


In [3]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [4]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest RMSE: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [15]:
# Classifier
#https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html

bayes_cv_tuner = BayesSearchCV(
    estimator = LGBMRegressor(
        objective='regression',
        metric=rmse_2,
        n_jobs=-1,
        verbose=-1
    ),
    search_spaces = {
        'learning_rate': (0.01, .04, 'log-uniform'),
        'num_leaves': (1, 200),      
        'max_depth': (0, 50), #reduce to lower overfitting 
        'min_child_samples': (0, 100),
        'max_bin': (100, 1000),
        'subsample': (0.2, .8, 'uniform'),
        'subsample_freq': (0, 10),
        'colsample_bytree': (0.3, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'subsample_for_bin': (100000, 500000),
        'reg_lambda': (1e-7, 1000, 'log-uniform'),
        'reg_alpha': (1e-7, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        'n_estimators': (50, 200),
        'num_leaves' : (20, 50),
        'min_data_in_leaf':(10, 40)
        "bagging_fraction" : (0.3, 1),
        "feature_fraction" : (0.3, 1),
        "bagging_frequency" : (2, 15),
    },    
    scoring = rmse_2,
    cv = KFold(
        n_splits=3,
        shuffle=True,
        random_state=42,
    ),
    n_jobs = -1,
    n_iter = 200,   
    verbose = 0,
    refit = True,
    random_state = 42,
    optimizer_kwargs = {'base_estimator': 'RF'}
)

# Fit the model
result = bayes_cv_tuner.fit(X, y, callback=status_print)

Model #1
Best RMSE: -1.3818
Best params: {'colsample_bytree': 0.5870727711973196, 'learning_rate': 0.027424237075668472, 'max_bin': 643, 'max_depth': 37, 'min_child_samples': 3, 'min_child_weight': 9, 'min_data_in_leaf': 25, 'n_estimators': 133, 'num_leaves': 28, 'reg_alpha': 1.3528298497308441e-05, 'reg_lambda': 0.30054941257055134, 'scale_pos_weight': 0.060830282487222269, 'subsample': 0.27610029103751044, 'subsample_for_bin': 301713, 'subsample_freq': 9}

Model #2
Best RMSE: -1.3818
Best params: {'colsample_bytree': 0.5870727711973196, 'learning_rate': 0.027424237075668472, 'max_bin': 643, 'max_depth': 37, 'min_child_samples': 3, 'min_child_weight': 9, 'min_data_in_leaf': 25, 'n_estimators': 133, 'num_leaves': 28, 'reg_alpha': 1.3528298497308441e-05, 'reg_lambda': 0.30054941257055134, 'scale_pos_weight': 0.060830282487222269, 'subsample': 0.27610029103751044, 'subsample_for_bin': 301713, 'subsample_freq': 9}

Model #3
Best RMSE: -1.3798
Best params: {'colsample_bytree': 0.6113827586

Model #19
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.30254121507832027, 'learning_rate': 0.03107296776803305, 'max_bin': 264, 'max_depth': 43, 'min_child_samples': 70, 'min_child_weight': 6, 'min_data_in_leaf': 38, 'n_estimators': 133, 'num_leaves': 48, 'reg_alpha': 1.5483714099246353e-05, 'reg_lambda': 3.3569576962452788e-06, 'scale_pos_weight': 59.630778855002092, 'subsample': 0.68796647870789829, 'subsample_for_bin': 209284, 'subsample_freq': 5}

Model #20
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.30254121507832027, 'learning_rate': 0.03107296776803305, 'max_bin': 264, 'max_depth': 43, 'min_child_samples': 70, 'min_child_weight': 6, 'min_data_in_leaf': 38, 'n_estimators': 133, 'num_leaves': 48, 'reg_alpha': 1.5483714099246353e-05, 'reg_lambda': 3.3569576962452788e-06, 'scale_pos_weight': 59.630778855002092, 'subsample': 0.68796647870789829, 'subsample_for_bin': 209284, 'subsample_freq': 5}

Model #21
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.302

Model #37
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.30254121507832027, 'learning_rate': 0.03107296776803305, 'max_bin': 264, 'max_depth': 43, 'min_child_samples': 70, 'min_child_weight': 6, 'min_data_in_leaf': 38, 'n_estimators': 133, 'num_leaves': 48, 'reg_alpha': 1.5483714099246353e-05, 'reg_lambda': 3.3569576962452788e-06, 'scale_pos_weight': 59.630778855002092, 'subsample': 0.68796647870789829, 'subsample_for_bin': 209284, 'subsample_freq': 5}

Model #38
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.30254121507832027, 'learning_rate': 0.03107296776803305, 'max_bin': 264, 'max_depth': 43, 'min_child_samples': 70, 'min_child_weight': 6, 'min_data_in_leaf': 38, 'n_estimators': 133, 'num_leaves': 48, 'reg_alpha': 1.5483714099246353e-05, 'reg_lambda': 3.3569576962452788e-06, 'scale_pos_weight': 59.630778855002092, 'subsample': 0.68796647870789829, 'subsample_for_bin': 209284, 'subsample_freq': 5}

Model #39
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.302

Model #55
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.30254121507832027, 'learning_rate': 0.03107296776803305, 'max_bin': 264, 'max_depth': 43, 'min_child_samples': 70, 'min_child_weight': 6, 'min_data_in_leaf': 38, 'n_estimators': 133, 'num_leaves': 48, 'reg_alpha': 1.5483714099246353e-05, 'reg_lambda': 3.3569576962452788e-06, 'scale_pos_weight': 59.630778855002092, 'subsample': 0.68796647870789829, 'subsample_for_bin': 209284, 'subsample_freq': 5}

Model #56
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.30254121507832027, 'learning_rate': 0.03107296776803305, 'max_bin': 264, 'max_depth': 43, 'min_child_samples': 70, 'min_child_weight': 6, 'min_data_in_leaf': 38, 'n_estimators': 133, 'num_leaves': 48, 'reg_alpha': 1.5483714099246353e-05, 'reg_lambda': 3.3569576962452788e-06, 'scale_pos_weight': 59.630778855002092, 'subsample': 0.68796647870789829, 'subsample_for_bin': 209284, 'subsample_freq': 5}

Model #57
Best RMSE: -1.3623
Best params: {'colsample_bytree': 0.302

Model #73
Best RMSE: -1.3585
Best params: {'colsample_bytree': 0.33199398274463543, 'learning_rate': 0.032076883022819455, 'max_bin': 700, 'max_depth': 24, 'min_child_samples': 88, 'min_child_weight': 6, 'min_data_in_leaf': 17, 'n_estimators': 154, 'num_leaves': 49, 'reg_alpha': 7.7923513483899033e-07, 'reg_lambda': 0.090090596600113135, 'scale_pos_weight': 14.35809754820448, 'subsample': 0.61050444134453852, 'subsample_for_bin': 282764, 'subsample_freq': 0}

Model #74
Best RMSE: -1.3585
Best params: {'colsample_bytree': 0.33199398274463543, 'learning_rate': 0.032076883022819455, 'max_bin': 700, 'max_depth': 24, 'min_child_samples': 88, 'min_child_weight': 6, 'min_data_in_leaf': 17, 'n_estimators': 154, 'num_leaves': 49, 'reg_alpha': 7.7923513483899033e-07, 'reg_lambda': 0.090090596600113135, 'scale_pos_weight': 14.35809754820448, 'subsample': 0.61050444134453852, 'subsample_for_bin': 282764, 'subsample_freq': 0}

Model #75
Best RMSE: -1.3585
Best params: {'colsample_bytree': 0.3319939

...

Model #127
Best RMSE: -1.358
Best params: {'colsample_bytree': 0.31449882335859358, 'learning_rate': 0.031826707937031282, 'max_bin': 236, 'max_depth': 31, 'min_child_samples': 100, 'min_child_weight': 4, 'min_data_in_leaf': 10, 'n_estimators': 133, 'num_leaves': 43, 'reg_alpha': 0.0045956549340442398, 'reg_lambda': 0.0028706328234893454, 'scale_pos_weight': 0.0028571097294120057, 'subsample': 0.74972257510738949, 'subsample_for_bin': 347676, 'subsample_freq': 2}

Model #128
Best RMSE: -1.358
Best params: {'colsample_bytree': 0.31449882335859358, 'learning_rate': 0.031826707937031282, 'max_bin': 236, 'max_depth': 31, 'min_child_samples': 100, 'min_child_weight': 4, 'min_data_in_leaf': 10, 'n_estimators': 133, 'num_leaves': 43, 'reg_alpha': 0.0045956549340442398, 'reg_lambda': 0.0028706328234893454, 'scale_pos_weight': 0.0028571097294120057, 'subsample': 0.74972257510738949, 'subsample_for_bin': 347676, 'subsample_freq': 2}

Model #129
Best RMSE: -1.358
Best params: {'colsample_bytree':

Model #145
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #146
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #147
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48

Model #163
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #164
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #165
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48

Model #181
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #182
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #183
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48

Model #199
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}

Model #200
Best RMSE: -1.3572
Best params: {'colsample_bytree': 0.48018395169871853, 'learning_rate': 0.019944787106672822, 'max_bin': 349, 'max_depth': 0, 'min_child_samples': 2, 'min_child_weight': 5, 'min_data_in_leaf': 33, 'n_estimators': 194, 'num_leaves': 50, 'reg_alpha': 1.3428918202554421e-06, 'reg_lambda': 0.36738052550567774, 'scale_pos_weight': 0.0077999778087354376, 'subsample': 0.72080408790270512, 'subsample_for_bin': 413393, 'subsample_freq': 8}



In [16]:
bayes_cv_tuner.score(X, y)

-1.0136453060501536

In [17]:
print(bayes_cv_tuner.total_iterations)

3000


In [ ]:
#https://scikit-optimize.github.io/notebooks/hyperparameter-optimization.html

In [ ]:
#XGboost
#https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769 

In [ ]:
COMPONENTS = 10

# List of decomposition methods to use
methods = [
    TruncatedSVD(n_components=COMPONENTS),
    PCA(n_components=COMPONENTS),
    FastICA(n_components=COMPONENTS),
    GaussianRandomProjection(n_components=COMPONENTS, eps=0.1),
    SparseRandomProjection(n_components=COMPONENTS, dense_output=True)    
]

# Run all the methods
embeddings = []
for method in methods:
    name = method.__class__.__name__    
    embeddings.append(
        pd.DataFrame(method.fit_transform(X), columns=[f"{name}_{i}" for i in range(COMPONENTS)])
    )
    print(f">> Ran {name}")
    
# Put all components into one dataframe
components_df = pd.concat(embeddings, axis=1)